In [1]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from skimage.io import imread
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import pandas as pd

In [2]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
K.clear_session()

In [4]:
data_entrenamiento = './train'
data_validacion = './test'

In [5]:
#Parametros red neuronal

epocas = 20
altura, longitud = 100, 100
batch_size = 32
pasos = 200
pasos_validacion = 20
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 29
lr = 0.0005

In [6]:
images_df = pd.read_csv(r'.\\file_target.csv')

images_df = images_df.rename(columns={'archivo': 'filenames',
                       'target': 'category'})

images_df.head()

,filenames,category
0,0.jpg,G
1,1.jpg,H
2,2.jpg,G
3,3.jpg,X
4,4.jpg,H


In [7]:
entrenamiento, validacion = train_test_split(images_df, test_size = 0.19)

entrenamiento = entrenamiento.reset_index(drop=True)

validacion = validacion.reset_index(drop=True)

In [8]:
#Pre-procesamiento de las imágenes
#Primero creamos un generador que dice como vamos a preprocesar la info

entrenamiento_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                          shear_range = 0.3,
                                          zoom_range = 0.3,
                                          horizontal_flip = True)

validacion_datagen = ImageDataGenerator(rescale = 1.0/255.0)

imagen_entrenamiento = entrenamiento_datagen.flow_from_dataframe(entrenamiento,
                                                                 data_entrenamiento,
                                                                 target_size=(altura, longitud),
                                                                 x_col='filenames',
                                                                y_col='category',
                                                                batch_size=batch_size,
                                                                class_mode='categorical')

imagen_validacion = validacion_datagen.flow_from_dataframe(validacion,
                                                           data_entrenamiento,
                                                          target_size=(altura, longitud),
                                                           x_col='filenames',
                                                            y_col='category',
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

Found 7290 validated image filenames belonging to 29 classes.
Found 1710 validated image filenames belonging to 29 classes.


In [9]:
cnn=Sequential()

cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding='same', input_shape=(altura, longitud, 3), activation='relu'))

cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding='same', activation='relu'))

cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())

cnn.add(Dense(256, activation='relu'))

cnn.add(Dropout(0.5))

cnn.add(Dense(clases, activation='softmax'))

In [10]:
cnn.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics=['accuracy'])

In [11]:
cnn.fit(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=pasos_validacion)

dir = './modelo/'

if not os.path.exists(dir):
    os.mkdir(dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

Epoch 1/20
200/200 [==============================] - 29s 142ms/step - loss: 3.4216 - accuracy: 0.0393 - val_loss: 3.2863 - val_accuracy: 0.0672
Epoch 2/20
200/200 [==============================] - 29s 144ms/step - loss: 3.2733 - accuracy: 0.0709 - val_loss: 2.9886 - val_accuracy: 0.1531
Epoch 3/20
200/200 [==============================] - 29s 146ms/step - loss: 3.0555 - accuracy: 0.1179 - val_loss: 2.6608 - val_accuracy: 0.2359
Epoch 4/20
200/200 [==============================] - 27s 135ms/step - loss: 2.8582 - accuracy: 0.1713 - val_loss: 2.4095 - val_accuracy: 0.3078
Epoch 5/20
200/200 [==============================] - 27s 134ms/step - loss: 2.6598 - accuracy: 0.1915 - val_loss: 2.1511 - val_accuracy: 0.3375
Epoch 6/20
200/200 [==============================] - 27s 133ms/step - loss: 2.5113 - accuracy: 0.2387 - val_loss: 2.1066 - val_accuracy: 0.3531
Epoch 7/20
200/200 [==============================] - 27s 134ms/step - loss: 2.4189 - accuracy: 0.2601 - val_loss: 1.9459 - val_ac